## An example to preprocess the Worldview-2 satellite image used in practical 2 (part 1) with the Geospatial Data Abstraction Library (GDAL)

* This Jupyter notebook contain code to 1) display metadata, 2) subset spatially (via .shp) and 3) spectrally (remove spectral bands) a 8 band Worldview-2 satellite image (Dunwich_20140701_WV2_raw.tif). I will include more code for band algebra (band ratios) and classification soon to replicate the image processing operations GEOM 2000/7000 students developed in Pract 2.  


### Initial setup

In [ ]:
## importing python modules needed to run the notebook

import pprint
from matplotlib import pyplot as plt
import numpy as np
from osgeo import gdal
import os

* Using [os.getcwd()](https://docs.python.org/3/library/os.html#) to check the working directory I am currently in

In [ ]:
## first to check the working directory

os.getcwd()


* In my case I am in C drive. In case I want to map where my image is I need to use os.chdir() in python as below:

In [ ]:
os.chdir('C:\\Users\\uqrborre\\Desktop\\RSRC_repo\\Data')

* Using the linux command "ls" to list what it is in 'C:\\Users\\uqrborre\\Desktop\\RSRC_repo\\Data'

In [ ]:
ls

* As you see the Dunwich_20140701_WV2_raw.tif image is in this location. Let's assign the image to the variable "filename" for further processing

In [ ]:
filename ='Dunwich_20140701_WV2_raw.tif'

### Quality assurance/checking routines

* You may want to assess the histogram per image band to check its shape (distribution) to be sure not many peaks are present (peaks may be a signal of data corruption)

In [ ]:
## lets get the image openned with Gdal first
image_gdal=gdal.Open(filename)

In [ ]:
## get band 5 from the image
image_gdal =image_gdal.GetRasterBand(5).ReadAsArray()

In [ ]:
## histogram for band 5
plt.hist(image_gdal.ravel(), density=True,  facecolor='b', alpha=0.75)
plt.xlabel('Pixel values')
plt.ylabel('Pixel number')
plt.title('Histogram for spectral band 5 (WV2 image)')
plt.grid(True)
plt.show()

* Lets's check the metadata for the image as the first step to assess its quality so to be sure that your image has all the bands, geographic attributes like datum, projection, coordinates system, extent etc. For this we use the GDAL tool "gdal info". First let's import the python module subprocess

In [ ]:
import subprocess
print (subprocess.check_output('gdalinfo '+'Dunwich_20140701_WV2_raw.tif'+' -norat'+' -nomd',shell=True))

* Let's have a look at the image (loading the second spectral band) in the jupyter notebook to be sure it is intact using python's [matplotlyb library](https://matplotlib.org/) 


In [ ]:
image_gdal =image.GetRasterBand(5).ReadAsArray()

In [ ]:
plt.imshow(image_gdal, cmap = "gray",  aspect='equal')

plt.colorbar()
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.title('Band 5 (WV2 image)')
plt.grid(True)

plt.show()


### Subsetting by shapefile

* A shapefile was previosly created (i.e. shape.shp located where the image is) to crop the image to include only land areas. The next code will use [gdalwarp](https://www.gdal.org/gdalwarp.html) to subset the image spatially 

In [ ]:
def subset(shp, img):
    outimg='Dunwich_20140701_WV2_raw8cut'+'.tif'
    cmd="gdalwarp -of GTiff -cutline %s -crop_to_cutline -dstalpha %s  %s" % (shp, img, outimg)
    os.system(cmd)


In [ ]:
shape='shape.shp'

In [ ]:
subset(shape, filename)

In [ ]:
imagecut=gdal.Open('Dunwich_20140701_WV2_raw8cut.tif')

In [ ]:
image_gdalcut =imagecut.GetRasterBand(5).ReadAsArray()

In [ ]:
#plt.imshow(image_gdalcut, vmin=0, vmax=255, cmap = "gray", aspect='auto')
plt.imshow(image_gdalcut, vmin=0, vmax=1500, cmap = "gray", aspect='equal')


#plt.colorbar()
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.title('Spatial subset band 5 (WV2 image)')
plt.grid(True)

plt.show()


## Spectral subsetting (i.e. remove spectral bands that may not be useful for classification)

* Next we will use the tool [gdal_translate](http://www.gdal.org/gdal_translate.html) to subset the image from 8 bands to 4 bands (RGBN). This will create a new image named "Dunwich_20140701_WV2_RGBN.tif"​

In [ ]:
def convertimg(infile):
    cmd= "gdal_translate -b 2 -b 3 -b 5 -b 7 %s Dunwich_20140701_WV2_RGBN.tif" %(infile)
    os.system(cmd)

In [ ]:
convertimg(filename)

* Done! the Dunwich_20140701_WV2_RGBN.tif image was created. To be sure let's read the working dir again and check the metadata for the number of bands.

In [ ]:
ls

* Checking metadata

In [ ]:
import subprocess
print (subprocess.check_output('gdalinfo '+'Dunwich_20140701_WV2_RGBN.tif'+' -norat'+' -nomd',shell=True))

....The next function in this notebook will produce an NDVI image from the WV-2 image subset
..stay tunned!